In [1]:
import sys, tensorflow as tf
print(tf.__version__)


2025-12-25 18:20:22.505166: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-25 18:20:22.544972: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-25 18:20:22.546009: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-25 18:20:23.316804: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.0


In [2]:
import sys
sys.executable




'/home/deres/fake-news-detection/venv310/bin/python'

In [3]:
import pandas as pd
import numpy as np

import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



In [4]:
fake_df = pd.read_csv("../data/Fake.csv")
true_df = pd.read_csv("../data/True.csv")

print("Fake shape:", fake_df.shape)
print("True shape:", true_df.shape)


Fake shape: (23481, 4)
True shape: (21417, 4)


In [5]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
fake_df["label"] = 0   # Fake
true_df["label"] = 1  # True


In [8]:
df = pd.concat([fake_df, true_df], axis=0).reset_index(drop=True)
print("Combined shape:", df.shape)
df.head()


Combined shape: (44898, 5)


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [9]:
df = df[["text", "label"]]
df.head()


,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    df["text"],
    df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)


In [11]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\n', ' ', text)
    return text


In [12]:
X_train_clean = X_train.apply(clean_text)
X_test_clean = X_test.apply(clean_text)

X_train_clean.head()


37645    paris reuters  french bank societe generale on...
30390    winstonsalem nc reuters  north carolina govern...
18191    civil political discourse took a beating in we...
25384    reuters  new york and washington state on mond...
32622    orlando fla reuters  orlando nightclub killer ...
Name: text, dtype: object

In [13]:
vectorizer = TfidfVectorizer(
    stop_words="english",
    max_df=0.7
)

X_train_tfidf = vectorizer.fit_transform(X_train_clean)
X_test_tfidf = vectorizer.transform(X_test_clean)

X_train_tfidf.shape


(35918, 186176)

In [14]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    "SVM": LinearSVC()
}

results = []

for name, model in models.items():
    print(f"Training {name}...")
    
    model.fit(X_train_tfidf, y_train)
    preds = model.predict(X_test_tfidf)
    
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, preds),
        "Precision": precision_score(y_test, preds),
        "Recall": recall_score(y_test, preds),
        "F1-Score": f1_score(y_test, preds)
    })

results_df = pd.DataFrame(results)
results_df

Training Logistic Regression...
Training Random Forest...
Training SVM...


,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.988530,0.986276,0.989729,0.988000
1,Random Forest,0.992539,0.989325,0.995098,0.992203
2,SVM,0.994989,0.993481,0.996032,0.994755


## LSTM
This section implements an LSTM-based neural network model for text classification.

In [15]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [16]:
print(type(X_train_clean))
print(len(X_train_clean))


<class 'pandas.core.series.Series'>
35918


In [17]:
max_words = 10000
max_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_clean)


X_train_seq = tokenizer.texts_to_sequences(X_train_clean)
X_test_seq = tokenizer.texts_to_sequences(X_test_clean)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding="post")



lstm_model = Sequential([
    Input(shape=(max_len,)),
    Embedding(input_dim=max_words, output_dim=128),
    LSTM(64),
    Dense(1, activation="sigmoid")
])

lstm_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

lstm_model.summary()



2025-12-25 18:21:21.209688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-25 18:21:21.210183: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 128)          1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
history = lstm_model.fit(
    X_train_pad,
    y_train,
    epochs=3,
    batch_size=64,
    validation_split=0.2
)


Epoch 1/3
449/449 [==============================] - 88s 192ms/step - loss: 0.2800 - accuracy: 0.8950 - val_loss: 0.2955 - val_accuracy: 0.8842
Epoch 2/3
449/449 [==============================] - 84s 187ms/step - loss: 0.2012 - accuracy: 0.9331 - val_loss: 0.1400 - val_accuracy: 0.9609
Epoch 3/3
449/449 [==============================] - 84s 186ms/step - loss: 0.1263 - accuracy: 0.9625 - val_loss: 0.1855 - val_accuracy: 0.9401


In [19]:
lstm_loss, lstm_acc = lstm_model.evaluate(X_test_pad, y_test)
print("LSTM Accuracy:", lstm_acc)


281/281 [==============================] - 12s 43ms/step - loss: 0.1722 - accuracy: 0.9449
LSTM Accuracy: 0.9448775053024292


In [20]:
final_results = pd.DataFrame({
    "Model": ["Logistic Regression", "Random Forest", "SVM", "LSTM"],
    "Accuracy": [0.9985, 0.9925, 0.9949, lstm_acc]
})

final_results


,Model,Accuracy
0,Logistic Regression,0.988500
1,Random Forest,0.992200
2,SVM,0.994900
3,LSTM,0.944878


## BERT Model Training

This section contains the implementation of a BERT-based transformer model
for fake news classification using the HuggingFace Transformers library.


In [21]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text 


In [22]:
bert_preprocess = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
    name="bert_preprocess"
)

bert_encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",
    trainable=True,
    name="bert_encoder"
)


In [23]:
text_input = tf.keras.layers.Input(
    shape=(),
    dtype=tf.string,
    name="text"
)

preprocessed_text = bert_preprocess(text_input)
bert_outputs = bert_encoder(preprocessed_text)

cls_output = bert_outputs["pooled_output"]

x = tf.keras.layers.Dropout(0.3)(cls_output)
output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

bert_model = tf.keras.Model(
    inputs=text_input,
    outputs=output
)


In [24]:
bert_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

bert_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 bert_preprocess (KerasLaye  {'input_type_ids': (None,    0         ['text[0][0]']                
 r)                          128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [ ]:
history_bert = bert_model.fit(
    X_train_clean,  
    y_train,
    validation_data=(X_test_clean, y_test),
    epochs=2,
    batch_size=16
)


Epoch 1/2
 129/2245 [>.............................] - ETA: 4:27:01 - loss: 0.1124 - accuracy: 0.9511

In [ ]:
comparison_results = pd.DataFrame([
    {
        "Model": "Logistic Regression",
        "Accuracy": results_df.loc[results_df["Model"] == "Logistic Regression", "Accuracy"].values[0]
    },
    {
        "Model": "Random Forest",
        "Accuracy": results_df.loc[results_df["Model"] == "Random Forest", "Accuracy"].values[0]
    },
    {
        "Model": "SVM",
        "Accuracy": results_df.loc[results_df["Model"] == "SVM", "Accuracy"].values[0]
    },
    {
        "Model": "LSTM",
        "Accuracy": lstm_acc   
    },
    {
        "Model": "BERT",
        "Accuracy": bert_acc
    }
])
comparison_results.sort_values(by="Accuracy", ascending=False)
